# Data Science Project Final Report
**By: Ricky Shi, Evan Chen, Mari Yasui, Shon Hoang**

---
# Introduction
### Data Description


This project explores the player behavior using a dataset collected by a UBC Computer Science research group that ran a Minecraft server. As players joined and interacted with the world, the server recorded their individual information. This includes their age, name, gender, how many hours they played in total, their experience level, and whether they were subscribed to a game-related newsletter. 


This project uses two datasets, `players.csv` and `sessions.csv`, collected from a Minecraft research server operated by UBC’s Computer Science department. The data record player demographics and in-game activity. My analysis will primarily use `players.csv`, as it includes both predictors and the response variable.

| Dataset | Rows | Columns | Description |
|----------|------|----------|--------------|
| `players.csv` | 196 | 7 | Contains hashed emails (acts as player IDs) and variables such as age, gender, total playtime (hours), and newsletter subscription status |
| `sessions.csv` | 1535 | 5 | Contains session-level data, including hashed emails, start and end times, and timestamp equivalents |

**Potential Issues**
- Missing age for two players reduces usable data
- Extreme playtime values (e.g., idle sessions) may report inaccurate playtimes
- Subscription classes are imbalanced (many more “Yes” than “No”)
- Predictors use different scales and must be standardized

In [3]:
library(tidyverse)
library(tidymodels)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

## Data Summary

In [4]:
players <- read_csv("https://github.com/FunkyMonkey245/dsci_project_planning_individual/raw/refs/heads/main/data/players.csv", show_col_types = FALSE)

sessions <- read_csv("https://github.com/FunkyMonkey245/dsci_project_planning_individual/raw/refs/heads/main/data/sessions.csv", show_col_types = FALSE)

players_summary <- players |>
    summarize(
        total_players = n(),
        subscribed_players = sum(subscribe == TRUE, na.rm = TRUE),
        unsubscribed_players = sum(subscribe == FALSE, na.rm = TRUE),
        subscribed_percent = round(100 * subscribed_players / total_players, 2),
        age_mean = round(mean(Age, na.rm = TRUE), 2),
        age_min  = min(Age, na.rm = TRUE),
        age_max  = max(Age, na.rm = TRUE),
        hours_mean = round(mean(played_hours, na.rm = TRUE), 2),
        hours_min  = min(played_hours, na.rm = TRUE),
        hours_max  = max(played_hours, na.rm = TRUE)
      )

sessions_summary <- sessions |>
    count(hashedEmail, name = "session_count") |>
    summarize(
        total_players = n(),
        total_sessions = sum(session_count),
        avg_sessions_per_player = mean(session_count),
        lowest_session_count = min(session_count),
        most_session_count = max(session_count)
      )
glimpse(players)
glimpse(sessions)
players_summary
sessions_summary

Rows: 196
Columns: 7
$ experience   <chr> "Pro", "Veteran", "Veteran", "Amateur", "Regular", "Amate…
$ subscribe    <lgl> TRUE, TRUE, FALSE, TRUE, TRUE, TRUE, TRUE, FALSE, TRUE, T…
$ hashedEmail  <chr> "f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8…
$ played_hours <dbl> 30.3, 3.8, 0.0, 0.7, 0.1, 0.0, 0.0, 0.0, 0.1, 0.0, 1.6, 0…
$ name         <chr> "Morgan", "Christian", "Blake", "Flora", "Kylie", "Adrian…
$ gender       <chr> "Male", "Male", "Male", "Female", "Male", "Female", "Fema…
$ Age          <dbl> 9, 17, 17, 21, 21, 17, 19, 21, 47, 22, 23, 17, 25, 22, 17…
Rows: 1,535
Columns: 5
$ hashedEmail         <chr> "bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8a…
$ start_time          <chr> "30/06/2024 18:12", "17/06/2024 23:33", "25/07/202…
$ end_time            <chr> "30/06/2024 18:24", "17/06/2024 23:46", "25/07/202…
$ original_start_time <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+12…
$ original_end_time   <dbl> 1.71977e+12, 1.71867e+12, 1.72193e+12, 1.72188e+

total_players,subscribed_players,unsubscribed_players,subscribed_percent,age_mean,age_min,age_max,hours_mean,hours_min,hours_max
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
196,144,52,73.47,21.14,9,58,5.85,0,223.1


total_players,total_sessions,avg_sessions_per_player,lowest_session_count,most_session_count
<int>,<int>,<dbl>,<int>,<int>
125,1535,12.28,1,310


**For *players* (players.csv), this data contains a list of 196 players (observations) which includes their 7 self-reported descriptions (variables):**

| Variable | Type | Description |
| --- | --- | --- |
| experience | factor | self-reported experience level (Amateur, Veteran, Regular, Behinner, Pro) |
| subscribe | logical | whether the player is subscribed to the newsletter |
| hashedEmail | character | a unique anonymized ID for each player |
| played_hours | double | total hours played by the player |
| name | character | the player's first name |
| gender | factor | the player's gender |
| Age | integer | the player's age |

The variables *experience and gender* have been changed to categorical variables to better aid in further data-visualization, while *Age* has been changed to an integer variable.


**Summary Report: `players.csv`**

- Total players: 196
- Subscribed players: 144
- Unsubscribed players: 52
- Percent Subscribed: 73.47%
- Average player age: 21.14 years
- Youngest player: 9 years
- Oldest player: 58 years
- Average total playtime: 5.85 hours
- Lowest playtime: 0 Hours
- Highest playtime: 223.10 Hours

## Establishing the Question

**Broad Question:**  
What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do they differ between various player types?

**Specific Question:**  
The specific question we aim to answer using this dataset is: Can a player’s age and total playtime predict whether they subscribe to the game’s newsletter? 
The goal of this question is to see whether a player’s age and total number of played hours are useful indicators of whether they choose to subscribe to the game’s newsletter. 

To answer this question, we use the `players.csv` dataset, which contains a row per player and several columns that describe their characteristics. The variables/columns we focus on are:
Subscribe (logical), which indicates whether the player is subscribed to the newsletter
Age (numeric), which indicates the player’s age in years
Played_hours, which indicates the total number of hours the player played on the server

## Exploratory Data Analysis and Visualization

- For this section, I will use the `players.csv` dataset to summarize the mean of the predictor variables, and visualize their relationships with newsletter subscription in multiple plots.
- Not much wrangling is needed as the dataset is already tidy for my analysis

In [3]:
# Mean summary of predictor variables
mean_summary <- players |>
    summarize(
        mean_age = round(mean(Age, na.rm = TRUE), 2),
        mean_playtime = round(mean(played_hours, na.rm = TRUE), 2),
      )

mean_summary

mean_age,mean_playtime
<dbl>,<dbl>
21.14,5.85


**Plots Included**
- Scatterplot of age vs. playtime colored by subscription
- Bar plot comparing subscription counts across experience levels
- Histograms showing playtime distribution for subscribers vs non-subscribers
- Log10 scaling was applied to playtime variables to compress extreme values and clearly show clustered data

**Observations**
- Most players recorded under 10 hours of total playtime
- There is a weak negative relationship between age and playtime, where younger players tend to play more
- Subscribed players generally show more total playtime
- Amateurs and veterans have the most subscribers

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 5)

#Data is already quite tidy, so changing the variable types is sufficient for my analysis

players <- players |>
    mutate(subscribe = as_factor(subscribe), subscribe = fct_recode(subscribe, "Yes" = "TRUE", "No" = "FALSE"), experience = as_factor(experience))

# Plot visualizations showing the relationships between multiple variables in the players dataset

players_plot <- ggplot(players, aes(x = Age, y = played_hours + 1, color = subscribe)) +
    geom_point(alpha = 0.7, size = 3) +
    scale_y_log10() +
    labs(x = "Age of Player (years)", y = "Total Playtime (hours)", color = "Subscribed") +
    ggtitle("Relationship Between Age, Playtime, and Newsletter Subscription") +
    theme(text = element_text(size = 14))

players_barplot <- ggplot(players, aes(x = experience, fill = subscribe)) +
    geom_bar(position = "dodge") +
    labs(title = "Newsletter Subscription by Experience Level", x = "Experience Level", y = "Proportion of Players", fill = "Subscribed") +
    theme(text = element_text(size = 14))

players_histogram <- ggplot(players, aes(x = played_hours, fill = subscribe)) +
    geom_histogram(bins = 20, alpha = 0.7, position = "identity") +
    facet_grid(rows = vars(subscribe)) +
    scale_x_log10() +
    labs(title = "Distribution of Total Playtime by Subscription Status", x = "Total Playtime (hours)", y = "Number of Players", fill = "Subscribed") +
    theme(text = element_text(size = 14))

players_barplot
players_histogram
players_plot 

Warning message in scale_x_log10():
“log-10 transformation introduced infinite values.”
Warning message:
“Removed 85 rows containing non-finite outside the scale range (`stat_bin()`).”


## Methods and Plan

I will use a K-NN classification model to predict subscription status using age and playtime.

**Why is this method appropriate?**

The research question is a binary classification problem (predicting subscription status [TRUE/FALSE]). The model predicts the outcome for a new player by finding the "k" closest players in the training data and using a majority vote on their subscription status. Unlike other methods, k-NN make sno assumptions about the underlying distribution of data, which is better when relationships might be complex.

**What assumptions are required?**

Some variables with larger ranges like played hours could dominate the distance calculation during k-NN classification, therefore requiring data normalization. The performance of k-NN decreases as the number of variables increase, however since we only have a few variables it would be managable, however if we decide to add more from the sessions.csv dataset the method may weaken. k-NN Classification also assumes that the distance represents the similarity between players (selected variables) are relevant to subscription status. 

**What are the potential weaknesses or limitations of this model?**

If more data is gathered, k-NN could be computationally expensive and slow as it must calculate the distance for every data point for a new prediction. If irrelevant variables are included, it could introduce unwanted noise to the method, reducing accuracy. 

**How are you going to compare and select the model?**

We can compare and select the model based on tuning for the best value of "k", implementing hyperparameter tuning, where 5-fold cross-validation will be used on the training dataset to maximize performance.

**How are you going to process the data to apply this model?**

The data will be first processed by splitting the players.csv dataset into a training set (75%) and a testing set (25%). Highly skewed data in played_hours and other variables will be standardized, while the missing values in the dataset (from the Age variable) could either be imputed using the median age (calculated by the training set) or by dropping the observations. Finally, the k-NN model will be trainied and the optimal "k" will be selected using k-fold cross-validation on the training set. The final model using the optimal "k" will be run on the testing set to report its final performance metrics.  

**Why this model?**
- It is suitable for categorical prediction using numerical predictors
- It does not require any particular relationship between the predictors and the outcome variables, but only uses nearby datapoints

**Assumptions & Preparation**
- Predictors will be standardized in a recipe using scaling and centering
- Missing ages will be filled using the mean age

**Limitations**
- Class imbalance (more subscribers) can bias results  
- Extreme points could overly influence distance-based calculations

**Model Selection & Evaluation**
- I will tune K (1–10) via 5-fold cross-validation using accuracy as the main metric
- Data is split into 70% training, 30% testing 
- The K with highest validation accuracy will be used to fit the final model and the accuracy on the test data will be used to answer the question

## Finding K

In [1]:
set.seed(2025)


players_split <- initial_split(players, prop = 0.70, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

players_recipe <- recipe(subscribe ~ Age, played_hours, data = players_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

players_spec <- nearest_neighbor(weight_func = "rectangular",
                              neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

players_vfold <- vfold_cv(players_train, v = 5, strata = subscribe)

players_wkflw <- workflow() |>
  add_recipe(players_recipe) |>
  add_model(players_spec)

players_wkflw

k_vals <- tibble(neighbors = seq(from = 1, to = 25, by = 1))

knn_results <- workflow() |>
  add_recipe(players_recipe) |>
  add_model(players_spec) |>
  tune_grid(resamples = players_vfold, grid = k_vals) |>
  collect_metrics()

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracies
Accuracy_graph <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate", title = "Number of Neighbors and their Accuracies") +
  theme(text = element_text(size = 12))

Accuracy_graph


ERROR: Error in initial_split(players, prop = 0.7, strata = subscribe): could not find function "initial_split"
